In [78]:
import pandas as pd
import numpy as np

df = pd.read_csv('agency.csv', header=0, index_col=[0,1])
df.replace('*', '', inplace=True)


#df.groupby(['Date', 'AssetClass'])['FNMATradeCount'].sum()
blah = df.loc[:, df.columns != 'Date'].apply(pd.to_numeric) 
df = pd.concat((blah, df['Date']), axis=1)

df.groupby(['Date']).sum()

,FNMATradeCount,FNMAUniqueID,FNMA$Trades,FHLMLCTradeCount,FHLMCUniqueID,FHLMLC$Trades,GNMATradeCount,GNMAUniqueID,GNMA$Trades,OtherTradeCount,OtherUniqueID,Other$Trades
Date,,,,,,,,,,,,
01/02/2019,7492.0,701.0,122176187.0,1197.0,472.0,14653660.5,1926.0,442.0,24679322.3,13.0,11.0,19188.3
01/03/2019,9897.0,818.0,183084731.7,1525.0,467.0,34840787.4,2218.0,436.0,56534687.9,20.0,15.0,27828.9
01/04/2019,7789.0,852.0,259730329.6,1470.0,471.0,46649005.6,2353.0,442.0,55255196.7,28.0,23.0,78487.7
01/07/2019,8574.0,1050.0,332869869.1,2006.0,583.0,79763619.1,2267.0,620.0,45199231.2,45.0,34.0,54710.8
01/08/2019,11515.0,1547.0,378607151.4,3727.0,1085.0,111353440.6,2605.0,537.0,65987482.4,62.0,42.0,325964.2
01/09/2019,11099.0,1406.0,292716668.2,2759.0,645.0,66816051.5,2275.0,568.0,54346610.9,29.0,16.0,306984.9
01/10/2019,9156.0,1227.0,194269761.4,2351.0,744.0,33790203.9,2958.0,707.0,69758038.2,32.0,22.0,102376.6
01/11/2019,6654.0,830.0,149208047.4,1841.0,727.0,28708820.8,2293.0,531.0,44425350.2,30.0,20.0,63075.2
01/14/2019,6559.0,870.0,118086797.1,1497.0,540.0,25541841.2,2257.0,596.0,47907799.5,25.0,15.0,12589.4


In [122]:
import pandas as pd

trading_date = '20190102'
trading_date = '{}/{}/{}'.format(trading_date[4:6], trading_date[6:], trading_date[:4])

price_fields = ['AVERAGE PRICE', 'WEIGHTED AVG. PRICE', 'AVG. PRICE BOTTOM 5 TRADES', '2ND QUARTILE PRICE',
                '3RD QUARTILE PRICE', '4TH QUARTILE PRICE', 'AVG. PRICE TOP 5 TRADES', 'STANDARD DEVIATION', 
                'VOLUME OF TRADES (000\'S)', 'NUMBER OF TRADES']

df = pd.read_excel('data/FINRA_IDS_PXTABLES-20190102.xlsx', sheet_name='TBA', skiprows=8, header=None)
df[0] = df[1].where(~df[1].isin(price_fields)).fillna(method='ffill')

#let's slice off the bottom part of this based on when the footnotes start - this is a variable line sheet
df = df.loc[:df[df[0].str.contains('Indicates')].index.values[0]-2]
df[df[0] == df[1]]

,0,1,2,3,4,5,6,7,8
0,"PRICING TABLE: AGENCY PASS-THRU (TBA, STIP, $ ...","PRICING TABLE: AGENCY PASS-THRU (TBA, STIP, $ ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,January Settlement,January Settlement,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Asset Sub-Class / Metric,Asset Sub-Class / Metric,COUPON,NaN,NaN,NaN,NaN,NaN,NaN
3,FNMA,FNMA,<= 2.5,3,3.5,4,4.5,5,> 5.0
14,FHLMC,FHLMC,<= 2.5,3,3.5,4,4.5,5,> 5.0
25,GNMA,GNMA,<= 2.5,3,3.5,4,4.5,5,> 5.0
37,"PRICING TABLE: AGENCY PASS-THRU (TBA, STIP, $ ...","PRICING TABLE: AGENCY PASS-THRU (TBA, STIP, $ ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,February Settlement,February Settlement,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,Asset Sub-Class / Metric,Asset Sub-Class / Metric,COUPON,NaN,NaN,NaN,NaN,NaN,NaN
40,FNMA,FNMA,<= 2.5,3,3.5,4,4.5,5,> 5.0
